# Challenge Owkin

N.B : Une présentation des mises en oeuvre des différentes méthodes est disponible dans l'annexe `lung_detail`

## Contexte

Owkin est une start-up française spécialisée dans le maching learning appliqué à la recherche médicale. Elle travaille à améliorer et à développer des solutions médicales en exploitant de grandes quantités de données, notamment par des méthodes de deep learning, dont Owkin possède une grande expertise.

## Objectif et description des données

Le but de ce data challenge est de développer des méthodes de prédiction du temps de survie de patients souffrant du cancer des poumons. Nous disposons pour chaque patient de données cliniques ainsi que de données quantitatives issues de la segmentation d'images provenant de radios de tumeurs cancéreuses. Ces dernières sont le fruit d'une étude de ces images par des experts et comprennent, par exemple, la couleur ou la densité des tumeurs. Ainsi, il s'agit d'un problème de régression sur des variables quantitatives.

Voici un exemple de fichier d'entrée :
|   |original_shape_Compactness1|original_shape_Compactness2| ... |original_glrlm_LongRunHighGrayLevelEmphasis|SurvivalTime|Event|
|---|---------------------------|---------------------------|-----|-------------------------------------------|------------|-----|
|002|0.024239                   |0.208753                   |     |6567.332842                                |638         |0    |
|003|0.014520                   |0.074913                   |     |901.956835                                 |421         |0    |
|004|0.022972                   |0.187502                   |     |3740.268350                                |465         |1    |
|005|0.030597                   |0.332627                   |     |10016.147961                               |1295        |1    |

Il s'agit de données censurées propres à l'analyse de survie et il n'est donc pas envisageable d'utiliser des méthodes statistiques classiques. Notre principale préoccupation est de mettre en oeuvre des méthodes adaptées à l'analyse de survie. On joint à ces données certaines données cliniques : l'age du patient (remplacé par la moyenne d'age lorsque cette donnée est manquante) et les variables Nstage et Tstage. Ces deux dernières variables proviennent de la classification TNM des tumeurs cancéreuses (voir https://fr.wikipedia.org/wiki/Classification_TNM).

Nous pouvons dans un premier temps chercher à éliminer les variables qui n'apportent pas d'information.

## Sélection des variables

Un rapide affichage des variances permet de détecter les variables ayant une très faible variance, ici classées de façon croissante.

|Variable                                        |Variance    |
|------------------------------------------------|------------|
|original_shape_Compactness1                     |3.860165e-05|
|original_glrlm_ShortRunLowGrayLevelEmphasis     |4.270926e-05|
|original_glrlm_LowGrayLevelRunEmphasis          |6.588227e-05|
|original_glcm_Idmn                              |1.106475e-04|
|original_glrlm_LongRunLowGrayLevelEmphasis      |6.257361e-04|
|original_glcm_Idn                               |1.049650e-03|
|original_glcm_JointEnergy                       |1.845187e-03|
|...                                             |...         |
|original_firstorder_Variance                    |1.296874e+09|
|original_shape_VoxelVolume                      |6.784925e+09|
|original_glcm_ClusterProminence                 |1.101009e+11|
|original_firstorder_Energy                      |2.594952e+19|

On a fait le choix d'éliminer les trois premières variables car les résultats étaient très satifaisants, mais on aurait pu en écarter davantage.
## Résultats

Nous avons mis en oeuvre différentes méthodes de machine learning adaptées à l'analyse de survie, ainsi que des méthodes plus élaborées de Deep Learning. Les premiers essais avec les machines à vecteurs support et les Random Survival Forests de `scikit-survival` n'ont pas été très concluants, mais le gradient boosting a donné quelques résultats assez encourageants. Nous avons pu atteindre un indice de concordance de 0.65 sur le jeu de données de test public, là où les précédentes méthodes n'ont pas dépassé 0.4.

Les méthodes du packages `lifelines` ont rapidement montré des résultats très intéressants. Le "penalized Cox's proportional hazard model" et le  "Weibull accelerated failure time model" ont respectivement abouti à des erreurs sur le jeu de données de test public de 0.723 et 0.724 dans les meilleurs des cas.

Pour terminer ce challenge, nous avons essayé d'appliquer des réseaux de neurones à ce problème. Encore une fois, il a fallu utiliser des méthodes adaptées à l'analyse de survie. Nous avons ainsi utilisé l'approche `DeepSurv`, qui est une adaptation du modèle de Cox aux réseaux de neurones, et qui est implémentée dans le packahe `pycox` (https://github.com/havakv/pycox). Dans un premier temps, cela s'est fait sur les données segmentées puis, dans un second temps, directement sur les images dont sont tirées ces données. Les résultats sont peu satisfaisants : 0.7 pour le modèle basé sur les données segmentés et 0.51 pour le second.